<img src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="400" align="right"/>

<br/>

# TELCO CHURN DEMO
# **PART 1: DATA ANALYSIS**

# 1. Data Upload
#### After using the DDL statements to load data in the Vantage Database, the next step is to create a connection to Vantage and begin examining the data.

# 2. Establish Vantage Connection

In [2]:
%lsconnect

Disconnected: NAME=demo, USER=vantage, HOST=tddb-env-d-268.vantage.demo.intellicloud.teradata.com


In [3]:
%addconnect NAME=demo, USER=vantage, HOST=tddb-env-d-268.vantage.demo.intellicloud.teradata.com

ERROR: Connection with the name 'demo' already exists

In [4]:
%connect demo

Password ·········


Success: 'demo' connection established


# 3. Data Discovery and Path Analysis (Vantage + Tableau)
#### 3a. Establish PATH table defining CHURN customers

In [6]:
DROP TABLE TelcoChurn.csi_telco_all_churn_paths;

Success: 17 rows affected

In [7]:
CREATE TABLE TelcoChurn.csi_telco_all_churn_paths
AS (SELECT *
    FROM NPATH
  (ON TelcoChurn.csi_telco_data
    PARTITION BY customerid
    ORDER BY tstamp
    USING
    MODE(NONOVERLAPPING)
    PATTERN('E*.C')
    SYMBOLS
    (event = 'CLOSED' AS C
    ,event <> 'CLOSED' AS E
    )
    RESULT
    (FIRST(customerid OF ANY(E,C)) AS CUSTOMERID
      ,COUNT(* OF E) AS EVENT_CNT
      ,ACCUMULATE(event OF ANY(E,C)) AS PATH
    )
  )
)
WITH DATA;

Success: 0 rows affected

In [8]:
SELECT
top 10 *
FROM TelcoChurn.csi_telco_all_churn_paths
ORDER BY event_cnt DESC;

customerid,event_cnt,path
350451,15,"[ACTIVATION, RATE PLAN CHANGE, DROPPED CALLS, CUSTOMER CARE, DROPPED CALLS, CUSTOMER CARE, DROPPED CALLS, CUSTOMER CARE, DROPPED CALLS, SUPPORT UNRESOLVED,"
351200,15,"[ACTIVATION, DROPPED CALLS, CUSTOMER CARE, SUPPORT ESCALATION, SUPPORT RESOLUTION, CUSTOMER CARE, SUPPORT RESOLUTION, CUSTOMER CARE, DROPPED CALLS, SUPPORT"
354516,11,"[ACTIVATION, DROPPED CALLS, CUSTOMER CARE, SUPPORT ESCALATION, SUPPORT UNRESOLVED, DROPPED CALLS, DROPPED CALLS, SUPPORT ESCALATION, DROPPED CALLS, SUPPORT"
350715,11,"[ACTIVATION, DROPPED CALLS, CUSTOMER CARE, SUPPORT ESCALATION, SUPPORT UNRESOLVED, RATE PLAN CHANGE, DROPPED CALLS, DROPPED CALLS, CUSTOMER CARE, SUPPORT E"
351045,10,"[ACTIVATION, DROPPED CALLS, CUSTOMER CARE, DROPPED CALLS, RATE PLAN CHANGE, SUPPORT ESCALATION, CUSTOMER CARE, SUPPORT UNRESOLVED, CUSTOMER CARE, CALL COMP"
350478,10,"[ACTIVATION, DROPPED CALLS, CUSTOMER CARE, DROPPED CALLS, CUSTOMER CARE, SUPPORT ESCALATION, CUSTOMER CARE, CALL COMPETITION, CALL COMPETITION, CALL COMPET"
351440,10,"[ACTIVATION, CUSTOMER CARE, SUPPORT UNRESOLVED, CUSTOMER CARE, RATE PLAN CHANGE, CUSTOMER CARE, SUPPORT UNRESOLVED, CALL COMPETITION, SUPPORT ESCALATION, C"
351062,10,"[ACTIVATION, DROPPED CALLS, DROPPED CALLS, CUSTOMER CARE, SUPPORT ESCALATION, SUPPORT RESOLUTION, DROPPED CALLS, CUSTOMER CARE, SUPPORT ESCALATION, SUPPORT"
351542,10,"[ACTIVATION, DROPPED CALLS, CUSTOMER CARE, DROPPED CALLS, SUPPORT ESCALATION, CALL COMPETITION, DROPPED CALLS, CUSTOMER CARE, SUPPORT UNRESOLVED, CALL COMP"
350149,9,"[ACTIVATION, DROPPED CALLS, CUSTOMER CARE, SUPPORT ESCALATION, CUSTOMER CARE, SUPPORT ESCALATION, CUSTOMER CARE, SUPPORT UNRESOLVED, CALL COMPETITION, CLOS"


#### 3b. Use the results of a custom CHURN paths query and the Teradata **Sankey Extension in Tableau**

#### 3c. Establish PATH table defining NON-CHURN customers 

In [9]:
DROP TABLE csi_telco_data_non_churners;

Success: 22 rows affected

In [10]:
CREATE TABLE csi_telco_data_non_churners
AS
(SELECT *
  FROM TelcoChurn.csi_telco_data
  WHERE customerid NOT IN (
      SELECT
      DISTINCT customerid 
      FROM TelcoChurn.csi_telco_all_churn_paths)
  AND tstamp < (SELECT
                  max(tstamp)
                  FROM TelcoChurn.csi_telco_data
                  WHERE event ='CLOSED')
)
WITH DATA;

Success: 0 rows affected

In [11]:
DROP TABLE TelcoChurn.csi_telco_all_nonchurn_paths;

Success: 17 rows affected

In [12]:
CREATE TABLE TelcoChurn.csi_telco_all_nonchurn_paths
AS
(SELECT *
    FROM NPATH
  (ON  
    csi_telco_data_non_churners
    PARTITION BY customerid 
    ORDER BY tstamp
    USING
    MODE(NONOVERLAPPING)
    PATTERN('E*')
    SYMBOLS
    (event <> 'CHURN' AS E)
    RESULT
    (FIRST(customerid OF ANY(E)) AS CUSTOMERID
      ,COUNT(* OF E) AS EVENT_CNT
      ,ACCUMULATE(event OF ANY(E)) AS PATH
    )
  ) 
)WITH DATA ;

Success: 0 rows affected

In [13]:
SELECT
TOP 10 *
FROM TelcoChurn.csi_telco_all_nonchurn_paths
ORDER BY event_cnt DESC;

customerid,event_cnt,path
637358,28,"[ACTIVATION, RATE PLAN CHANGE, SUPPORT RESOLUTION, BILL OVERAGES, BILL OVERAGES, SUPPORT RESOLUTION, RATE PLAN CHANGE, SUPPORT RESOLUTION, RETAIL VISIT, BI"
403431,27,"[ACTIVATION, RATE PLAN CHANGE, CUSTOMER CARE, RATE PLAN CHANGE, BILL OVERAGES, CUSTOMER CARE, BILL OVERAGES, RETAIL VISIT, SUPPORT RESOLUTION, BILL OVERAGE"
469205,27,"[ACTIVATION, RATE PLAN CHANGE, BILL OVERAGES, CUSTOMER CARE, RETAIL VISIT, CUSTOMER CARE, SUPPORT RESOLUTION, BILL OVERAGES, BILL OVERAGES, CUSTOMER CARE,"
538696,26,"[ACTIVATION, SUPPORT RESOLUTION, CUSTOMER CARE, CUSTOMER CARE, RATE PLAN CHANGE, SUPPORT RESOLUTION, RATE PLAN CHANGE, RETAIL VISIT, CUSTOMER CARE, RETAIL"
537296,26,"[ACTIVATION, SUPPORT RESOLUTION, SUPPORT RESOLUTION, RATE PLAN CHANGE, BILL OVERAGES, SUPPORT RESOLUTION, CUSTOMER CARE, BILL OVERAGES, SUPPORT RESOLUTION,"
438384,26,"[ACTIVATION, CUSTOMER CARE, BILL OVERAGES, RETAIL VISIT, RETAIL VISIT, RETAIL VISIT, CUSTOMER CARE, SUPPORT RESOLUTION, SUPPORT RESOLUTION, RATE PLAN CHANG"
380332,26,"[ACTIVATION, SUPPORT RESOLUTION, CUSTOMER CARE, CUSTOMER CARE, CUSTOMER CARE, RETAIL VISIT, CUSTOMER CARE, RETAIL VISIT, SUPPORT RESOLUTION, RETAIL VISIT,"
350774,26,"[ACTIVATION, SUPPORT RESOLUTION, CUSTOMER CARE, CUSTOMER CARE, RATE PLAN CHANGE, SUPPORT RESOLUTION, SUPPORT RESOLUTION, CUSTOMER CARE, RATE PLAN CHANGE, S"
599175,26,"[ACTIVATION, BILL OVERAGES, BILL OVERAGES, RATE PLAN CHANGE, BILL OVERAGES, SUPPORT RESOLUTION, RETAIL VISIT, BILL OVERAGES, CUSTOMER CARE, CUSTOMER CARE,"
437930,26,"[ACTIVATION, CUSTOMER CARE, CUSTOMER CARE, RETAIL VISIT, RATE PLAN CHANGE, RATE PLAN CHANGE, CUSTOMER CARE, RATE PLAN CHANGE, CUSTOMER CARE, RATE PLAN CHAN"


# 4. Data Discovery Text (Vantage)
#### a. Use the Ngrams function to tokenize the text of CHURN and NON-CHURN

In [14]:
DROP TABLE TelcoChurn.csi_telco_all_churn_paths_token;

Success: 18 rows affected

In [15]:
CREATE MULTISET TABLE TelcoChurn.csi_telco_all_churn_paths_token
AS (
  SELECT *
    FROM NGrams (
    ON  TelcoChurn.csi_telco_all_churn_paths
    USING
    TextColumn ('path')
    Delimiter (',')
    Grams ('1')
    Overlapping ('false')
    ToLowerCase ('true')
    Punctuation ('\[.,?\!\]')
    Reset ('\[.,?\!\]')
    TotalGramCount ('false')
    Accumulate ('customerid')
  ) AS dt
)
WITH DATA;

Success: 0 rows affected

In [16]:
SELECT
TOP 10 *
FROM TelcoChurn.csi_telco_all_churn_paths_token
ORDER BY frequency DESC;

customerid,ngram,n,frequency
350451,dropped calls,1,5
354516,dropped calls,1,4
350451,customer care,1,4
351200,support escalation,1,4
351137,customer care,1,3
351062,dropped calls,1,3
351200,dropped calls,1,3
350478,customer care,1,3
352348,dropped calls,1,3
350478,call competition,1,3


In [17]:
SELECT
top 10 *
FROM TelcoChurn.csi_telco_all_churn_paths_token
WHERE customerid = 350451
ORDER BY frequency DESC;

customerid,ngram,n,frequency
350451,dropped calls,1,5
350451,customer care,1,4
350451,call competition,1,2
350451,support unresolved,1,2
350451,[activation,1,1
350451,closed],1,1
350451,rate plan change,1,1


In [18]:
DROP TABLE TelcoChurn.csi_telco_all_nonchurn_paths_token;

Success: 18 rows affected

In [19]:
CREATE MULTISET TABLE TelcoChurn.csi_telco_all_nonchurn_paths_token
AS (SELECT *
    FROM NGrams (
    ON  TelcoChurn.csi_telco_all_nonchurn_paths
    USING
    TextColumn ('path')
    Delimiter (',')
    Grams ('1')
    Overlapping ('false')
    ToLowerCase ('true')
    Punctuation ('\[.,?\!\]')
    Reset ('\[.,?\!\]')
    TotalGramCount ('false')
    Accumulate ('customerid')
  ) AS dt
)
WITH DATA;

Success: 0 rows affected

In [20]:
SELECT
TOP 10 *
FROM TelcoChurn.csi_telco_all_nonchurn_paths_token
ORDER BY frequency DESC;

customerid,ngram,n,frequency
449708,customer care,1,13
671895,customer care,1,12
363769,customer care,1,12
501454,customer care,1,12
531558,customer care,1,12
578908,customer care,1,12
640253,customer care,1,12
544593,customer care,1,12
520941,customer care,1,12
371650,customer care,1,12


In [21]:
SELECT
top 10 *
FROM TelcoChurn.csi_telco_all_nonchurn_paths_token
WHERE customerid = 449708
ORDER BY frequency DESC;

customerid,ngram,n,frequency
449708,customer care,1,13
449708,support resolution,1,3
449708,rate plan change,1,1
449708,bill overages,1,1
449708,[activation,1,1
449708,retail visit],1,1


#### 4b. Data Cleaning using OREPLACE

In [22]:
DROP TABLE TelcoChurn.csi_telco_all_churn_paths_token_clean; 

Success: 18 rows affected

In [23]:
CREATE MULTISET TABLE TelcoChurn.csi_telco_all_churn_paths_token_clean
AS (SELECT 
  oreplace 
    (oreplace (ngram,'[',''), ']','')  AS ngram 
    , customerid
    , n
    , frequency 
    FROM TelcoChurn.csi_telco_all_churn_paths_token
) 
WITH DATA;

Success: 0 rows affected

In [24]:
SELECT
TOP 10 *
FROM TelcoChurn.csi_telco_all_churn_paths_token_clean
WHERE customerid = 350451
ORDER BY frequency DESC;

ngram,customerid,n,frequency
dropped calls,350451,1,5
customer care,350451,1,4
support unresolved,350451,1,2
call competition,350451,1,2
activation,350451,1,1
rate plan change,350451,1,1
closed,350451,1,1


In [25]:
DROP TABLE TelcoChurn.csi_telco_all_nonchurn_paths_token_clean;

Success: 18 rows affected

In [26]:
CREATE MULTISET TABLE TelcoChurn.csi_telco_all_nonchurn_paths_token_clean
AS (SELECT 
  oreplace 
  (oreplace (ngram,'[',''), ']','')  as ngram
    , customerid
    ,n
    , frequency
    FROM TelcoChurn.csi_telco_all_nonchurn_paths_token
)
WITH DATA;

Success: 0 rows affected

In [27]:
SELECT
TOP 10 *
FROM TelcoChurn.csi_telco_all_nonchurn_paths_token_clean
WHERE customerid = 449708
ORDER BY frequency DESC;

ngram,customerid,n,frequency
customer care,449708,1,13
support resolution,449708,1,3
rate plan change,449708,1,1
activation,449708,1,1
bill overages,449708,1,1
retail visit,449708,1,1


#### 5. Create a master Analytics Data Set (ADS) prior to modelling for all known PATHS

In [28]:
DROP TABLE TelcoChurn.model_dataset_raw;

Success: 17 rows affected

In [29]:
CREATE MULTISET TABLE TelcoChurn.model_dataset_raw
AS (
    SELECT
    raw.customerid
    , raw.token
    , raw.category
    FROM(
        SELECT
        customerid
        ,lower(ngram) AS token
        ,'NON CHURN' as category
        FROM TelcoChurn.csi_telco_all_nonchurn_paths_token_clean
        UNION ALL
        SELECT
        customerid
        ,lower(ngram) AS token,
        'CHURN' as category
        FROM TelcoChurn.csi_telco_all_churn_paths_token_clean
        ) raw
    )
WITH DATA;

Success: 0 rows affected

In [30]:
SELECT
TOP 10 *
FROM TelcoChurn.model_dataset_raw;

customerid,token,category
472787,activation,NON CHURN
472787,support resolution,NON CHURN
472787,bill overages,NON CHURN
447501,support resolution,NON CHURN
447501,customer care,NON CHURN
447501,activation,NON CHURN
447501,support resolution,NON CHURN
472787,customer care,NON CHURN
472787,customer care,NON CHURN
472787,rate plan change,NON CHURN
